In [47]:
import setup # resolve path to 'src'
import numpy as np
import pandas as pd

from typing import Optional
from build_parse import *
from metrics import *

pd.set_option('display.max_rows', None)

In [48]:
progs = [ CoreutilsProgram(progname) for progname in COREUTILS_PROG_NAMES ]
prognames = [ prog.get_name() for prog in progs ]

prognames_analyze = [ "stat", "nohup", "pinky", "csplit", "ginstall", "fmt", "df", "join", "expr", "seq", "unexpand", "tsort", "tee", "base64", "sum", "cksum", "wc" ]
progs_analyze = []
for progname in prognames_analyze:
    for prog in progs:
        if progname == prog.get_name():
            progs_analyze.append(prog)
            break

def prog_from_progname(progname: str) -> Program:
    for prog in progs:
        if progname == prog.get_name():
            return prog

# Define the build options to test for each program
debug_opts = BuildOptions(debug=True, strip=False, optimization=0)
standard_opts = BuildOptions(debug=False, strip=False, optimization=0)
strip_opts = BuildOptions(debug=False, strip=True, optimization=0)

opts_sets = (strip_opts, standard_opts, debug_opts)

# Get the parser functions
dwarf_parser = get_parser("dwarf")
ghidra_parser = get_parser("ghidra")

varnode_compare_levels = list(VarnodeCompareLevel.range())
varnode_compare_level_labels = [ VarnodeCompareLevel.to_string(level) for level in varnode_compare_levels ]

primitive_metatypes = [MetaType.INT, MetaType.FLOAT, MetaType.POINTER]
complex_metatypes = [MetaType.ARRAY, MetaType.STRUCT, MetaType.UNION]
metatypes = primitive_metatypes + complex_metatypes
metatype_labels = [ MetaType.repr(metatype) for metatype in metatypes ]

# ensure that each program is built according to all variations of build options
for prog in progs:
    for opts in (debug_opts, standard_opts, strip_opts):
        assert(prog.valid_build(opts))

In [49]:

# the filename format for saving parsed ProgramInfo pickle objects
def mangle_proginfo_save_name(parsername: str, prog: Program, opts: BuildOptions) -> str:
    return "{}.{}.pickle".format(prog.get_binary_name(opts), parsername)

def get_proginfo_save_path(parsername: str, prog: Program, opts: BuildOptions) -> Path:
    return PICKLE_CACHE_DIR.joinpath(mangle_proginfo_save_name(parsername, prog, opts))

def save_proginfo(proginfo: ProgramInfo, parsername: str, prog: Program, opts: BuildOptions):
    save_pickle(proginfo, get_proginfo_save_path(parsername, prog, opts))

def load_proginfo(parsername: str, prog: Program, opts: BuildOptions) -> ProgramInfo:
    return load_pickle(get_proginfo_save_path(parsername, prog, opts))

# the filename format for saving UnoptimizedProgramInfoCompare2 objects
def mangle_cmp_save_name(prog: Program, opts: BuildOptions) -> str:
    return "{}.cmp.pickle".format(prog.get_binary_name(opts))

def get_cmp_save_path(prog: Program, opts: BuildOptions) -> Path:
    return PICKLE_CACHE_DIR.joinpath(mangle_cmp_save_name(prog, opts))

def save_cmp(cmp: UnoptimizedProgramInfoCompare2, prog: Program, opts: BuildOptions):
    save_pickle(cmp, get_cmp_save_path(prog, opts))

def load_cmp(prog: Program, opts: BuildOptions) -> UnoptimizedProgramInfoCompare2:
    return load_pickle(get_cmp_save_path(prog, opts))

In [50]:
# DWARF: only parse with the debug build options
# Ghidra: parse with all variations of build options
# Cache the results in local pickle_cache directory, named based on the 'mangle' scheme

reparse = False # should we re-parse even if we already parsed and cached a program?
skip_parsing = True # should we skip the parsing? set to True if we already parsed & cached

class ParseException(Exception):
    pass

def parse(parser: Callable, prog: Program, opts: BuildOptions) -> Optional[ProgramInfo]:
    try:
        return parser(prog.get_binary_path(opts))
    except:
        return None

failed = []
if not skip_parsing:
    for prog in progs:
        dwarf_debug_savepath = get_proginfo_save_path("dwarf", prog, debug_opts)
        if reparse or not dwarf_debug_savepath.exists():
            dwarf_debug = parse(dwarf_parser, prog, debug_opts)
            if dwarf_debug is None:
                failed.append(("dwarf", prog.get_name(), debug_opts))
            else:
                save_pickle(dwarf_debug, dwarf_debug_savepath)

        for opts in opts_sets:
            ghidra_parse_savepath = get_proginfo_save_path("ghidra", prog, opts)
            if reparse or not dwarf_debug_savepath.exists():
                ghidra_parse = parse(ghidra_parser, prog, opts)
                if ghidra_parse is None:
                    failed.append(("ghidra", prog.get_name(), opts))
                else:
                    save_pickle(ghidra_parse, ghidra_parse_savepath)

In [51]:
print(failed)

for prog in progs:
    for opts in opts_sets:
        assert(get_proginfo_save_path("ghidra", prog, opts).exists())

[]


In [52]:
# For each program & build options combination, compute & store comparison object

recompare = False
skip_comparisons = True

failed = []
if not skip_comparisons:
    for prog in progs:
        # load the DWARF ground-truth ProgramInfo
        dwarf_proginfo = load_pickle(get_proginfo_save_path("dwarf", prog, debug_opts))
        assert(dwarf_proginfo is not None)

        # for each set of compilation options, load the Ghidra decompiler ProgramInfo
        # then compute & store the comparison object
        for opts in (strip_opts,):
            cmp_save_path = get_cmp_save_path(prog, opts)
            if recompare or not cmp_save_path.exists():
                ghidra_proginfo = load_pickle(get_proginfo_save_path("ghidra", prog, opts))
                assert(ghidra_proginfo is not None)
                try:
                    cmp = compare2(dwarf_proginfo, ghidra_proginfo)
                    save_pickle(cmp, get_cmp_save_path(prog, opts))
                except:
                    failed.append((prog.get_name(), opts))

In [53]:
print(failed)

for prog in progs:
    for opts in opts_sets:
        assert(get_cmp_save_path(prog, opts).exists())

[]


In [54]:
# For each opts, compute the tables

def mangle_table_save_name(
    tablename: str,
    opts: BuildOptions
) -> str:
    return "{}{}.csv".format(tablename, suffix(opts))

def build_options_display_suffix(opts: BuildOptions) -> str:
    return "(optimization={}, stripped={}, debug={})".format(opts.optimization, opts.strip, opts.debug)

def mangle_table_display_name(
    tablename: str,
    opts: BuildOptions
) -> str:

    return "{} {}".format(tablename, build_options_display_suffix(opts))

def get_table_save_path(
    tablename: str,
    opts: BuildOptions
) -> Path:
    return DATA_DIR.joinpath(mangle_table_save_name(tablename, opts))

def load_table(
    tablename: str,
    opts: BuildOptions
) -> pd.DataFrame:
    return pd.read_csv(get_table_save_path(tablename, opts), index_col=0)

def load_table_filter_analyzed(tablename: str, opts: BuildOptions) -> pd.DataFrame:
    return load_table(tablename, opts).filter(prognames_analyze, axis=0)

In [55]:
metrics_groups = make_metrics()

bytes_group = metrics_groups[0]
functions_group = metrics_groups[1]
varnodes_group = metrics_groups[2]
decomposed_varnodes_group = metrics_groups[9]
array_comparisons_group = metrics_groups[13]

def varnodes_group_metatype(metatype: int) -> MetricsGroup:
    _map = dict([ (meta, i) for i, meta in enumerate(primitive_metatypes + complex_metatypes, 3) ])
    return metrics_groups[_map[metatype]]

varnodes_groups_metatypes = [ varnodes_group_metatype(metatype) for metatype in (primitive_metatypes + complex_metatypes) ]

def decomposed_varnodes_group_metatype(metatype: int) -> MetricsGroup:
    _map = dict([ (meta, i) for i, meta in enumerate(primitive_metatypes, 10) ])
    return metrics_groups[_map[metatype]]

decomposed_varnodes_groups_metatypes = [ decomposed_varnodes_group_metatype(metatype) for metatype in primitive_metatypes ]

high_varnodes_groups = [varnodes_group] + varnodes_groups_metatypes
decomposed_varnodes_groups = [decomposed_varnodes_group] + decomposed_varnodes_groups_metatypes

In [56]:
recompute = False
skip_compute_metrics = True

if not skip_compute_metrics:
    for opts in opts_sets:
        cmps = [ load_cmp(prog, opts) for prog in progs ]
        for grp in metrics_groups:
            save_path = get_table_save_path(grp.get_name(), opts)
            tablename = mangle_table_display_name(grp.get_display_name(), opts)
            print(tablename)
            if recompute or not save_path.exists():
                df = compute_comparisons_metrics_dataframe(prognames, cmps, grp.get_metrics())
                df.to_csv(save_path)


In [57]:
def get_table(
    grp: MetricsGroup,
    opts: BuildOptions,
    analyzed_only: bool = True
)-> pd.DataFrame:
    df = load_table(grp.get_name(), opts)
    return df if not analyzed_only else df.filter(prognames_analyze, axis=0)

def display_analyzed_tables(
    metrics_groups: List[MetricsGroup],
    opts_sets: List[BuildOptions],
    analyzed_only: bool = True
):
    for grp in metrics_groups:
        for opts in opts_sets:
            table_display_name = mangle_table_display_name(grp.get_display_name(), opts)
            df = get_table(grp, opts, analyzed_only=analyzed_only)

            print("{} {} {}".format("-"*10, table_display_name, "-"*10))
            display(df)

In [58]:
skip_fix_varnode_metrics = False

# Add "Varnodes fraction partially recovered" & "Varnodes fraction exactly recovered" columns
# to the varnodes tables (if not already done)
if not skip_fix_varnode_metrics:
    for grp in high_varnodes_groups + decomposed_varnodes_groups:
        for opts in opts_sets:
            df = get_table(grp, opts, analyzed_only=False)
            df["Varnodes fraction partially recovered"] = df.iloc[:,2:6].sum(axis=1) / df.iloc[:,0]
            df["Varnodes fraction exactly recovered"] = df.iloc[:,5] / df.iloc[:,0]
            savepath = get_table_save_path(grp.get_name(), opts)
            df.to_csv(savepath)

def get_varnode_group_average_stats(grp: MetricsGroup, analyzed_only: bool = True) -> pd.Series:
    df = get_table(grp, opts, analyzed_only=analyzed_only)
    return df.iloc[:,6:].mean(axis=0)

In [59]:
skip_generate_metatype_level_summaries = True

if not skip_generate_metatype_level_summaries:
    for opts in opts_sets:
        for analyzed_only in (True, False):
            _suffix = "_analyzed_only" if analyzed_only else ""
            seriess = []
            for metatype in metatypes[:-1]:
                metatype_str = MetaType.repr(metatype)
                grp = varnodes_group_metatype(metatype)
                df = get_table(grp, opts, analyzed_only=analyzed_only)
                metatype_varnodes = df.iloc[:,0].sum()
                varnodes_by_levels = df.iloc[:,1:6].sum(axis=0)
                varnodes_by_levels.index = varnode_compare_level_labels
                level_ratios = varnodes_by_levels / metatype_varnodes
                seriess.append(level_ratios)
            
            high_df = pd.DataFrame(
                seriess,
                index=[ MetaType.repr(metatype) for metatype in metatypes[:-1] ],
                columns=varnode_compare_level_labels
            )
            high_tablename = "metatype_match_levels_ratios" + _suffix
            high_savepath = get_table_save_path(high_tablename, opts)
            print("{}{}".format(high_tablename, suffix(opts)))
            display(high_df)
            high_df.to_csv(high_savepath)

            decomposed_seriess = []
            for metatype in primitive_metatypes:
                metatype_str = MetaType.repr(metatype)
                grp = decomposed_varnodes_group_metatype(metatype)
                df = get_table(grp, opts, analyzed_only=analyzed_only)
                metatype_varnodes = df.iloc[:,0].sum()
                varnodes_by_levels = df.iloc[:,1:6].sum(axis=0)
                varnodes_by_levels.index = varnode_compare_level_labels
                level_ratios = varnodes_by_levels / metatype_varnodes
                decomposed_seriess.append(level_ratios)

            decomposed_df = pd.DataFrame(
                decomposed_seriess,
                index=[ MetaType.repr(metatype) for metatype in primitive_metatypes ],
                columns=varnode_compare_level_labels
            )
            decomposed_tablename = "metatype_match_levels_ratios_decomposed" + _suffix
            decomposed_savepath = get_table_save_path(decomposed_tablename, opts)
            print("{}{}".format(decomposed_tablename, suffix(opts)))
            display(decomposed_df)
            decomposed_df.to_csv(decomposed_savepath)

In [60]:
df = load_table("metatype_match_levels_ratios_analyzed_only", debug_opts)
df

,NO_MATCH,OVERLAP,SUBSET,ALIGNED,MATCH
INT,0.007421,0.000530,0.000000,0.00318,0.988868
FLOAT,0.000000,0.000000,0.000000,0.00000,1.000000
POINTER,0.009042,0.000000,0.000000,0.00000,0.990958
ARRAY,0.142857,0.002421,0.009685,0.00000,0.845036
STRUCT,0.000000,0.002959,0.000000,0.00000,0.997041


In [61]:
# Function Analysis

for opts in opts_sets:
    print(mangle_table_display_name(functions_group.get_name(), opts))
    display(get_table(functions_group, opts, analyzed_only=True))

functions (optimization=0, stripped=True, debug=False)


,Ground truth functions,Functions found,Functions missed,Functions recovery fraction
stat,240,240,0,1.0
nohup,115,115,0,1.0
pinky,124,124,0,1.0
csplit,339,339,0,1.0
fmt,131,131,0,1.0
df,266,266,0,1.0
join,152,152,0,1.0
expr,323,323,0,1.0
seq,129,129,0,1.0
unexpand,121,121,0,1.0


functions (optimization=0, stripped=False, debug=False)


,Ground truth functions,Functions found,Functions missed,Functions recovery fraction
stat,240,240,0,1.0
nohup,115,115,0,1.0
pinky,124,124,0,1.0
csplit,339,339,0,1.0
fmt,131,131,0,1.0
df,266,266,0,1.0
join,152,152,0,1.0
expr,323,323,0,1.0
seq,129,129,0,1.0
unexpand,121,121,0,1.0


functions (optimization=0, stripped=False, debug=True)


,Ground truth functions,Functions found,Functions missed,Functions recovery fraction
stat,240,240,0,1.0
nohup,115,115,0,1.0
pinky,124,124,0,1.0
csplit,339,339,0,1.0
fmt,131,131,0,1.0
df,266,266,0,1.0
join,152,152,0,1.0
expr,323,323,0,1.0
seq,129,129,0,1.0
unexpand,121,121,0,1.0


In [67]:
# Varnode Analysis

for opts in opts_sets:
    print("{} HIGH-LEVEL VARNODES ANALYSIS {} {}".format("-"*10, build_options_display_suffix(opts), "-"*10))
    
    for grp in high_varnodes_groups:
        print(mangle_table_display_name(grp.get_name(), opts))
        display(get_table(grp, opts))
        display(get_varnode_group_average_stats(grp, opts))
    
    display(load_table("metatype_match_levels_ratios_analyzed_only", opts))

---------- HIGH-LEVEL VARNODES ANALYSIS (optimization=0, stripped=True, debug=False) ----------
varnodes (optimization=0, stripped=True, debug=False)


,Ground truth varnodes,Varnodes matched @ level=NO_MATCH,Varnodes matched @ level=OVERLAP,Varnodes matched @ level=SUBSET,Varnodes matched @ level=ALIGNED,Varnodes matched @ level=MATCH,"Varnode average comparison score [0,1]",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,608,20,22,20,313,233,0.794819,0.967105,0.383224
nohup,162,5,8,6,102,41,0.756173,0.969136,0.253086
pinky,182,8,12,5,106,51,0.747253,0.956044,0.280220
csplit,1003,15,43,17,540,388,0.809821,0.985045,0.386839
fmt,186,6,8,4,107,61,0.780914,0.967742,0.327957
df,646,9,25,15,283,314,0.835913,0.986068,0.486068
join,260,9,12,5,145,89,0.781731,0.965385,0.342308
expr,932,15,38,27,496,356,0.805794,0.983906,0.381974
seq,279,8,21,8,121,121,0.792115,0.971326,0.433692
unexpand,158,5,7,5,89,52,0.778481,0.968354,0.329114


Varnode average comparison score [0,1]    0.778015
Varnodes fraction partially recovered     0.961087
Varnodes fraction exactly recovered       0.351030
dtype: float64

varnodes_metatype_INT (optimization=0, stripped=True, debug=False)


,Ground truth varnodes (metatype=INT),Decompiler varnodes matched @ level=NO_MATCH (metatype=INT),Decompiler varnodes matched @ level=OVERLAP (metatype=INT),Decompiler varnodes matched @ level=SUBSET (metatype=INT),Decompiler varnodes matched @ level=ALIGNED (metatype=INT),Decompiler varnodes matched @ level=MATCH (metatype=INT),"Varnode average compare score [0,1] (metatype=INT)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,392,4,1,0,233,154,0.839286,0.989796,0.392857
nohup,99,0,0,0,71,28,0.820707,1.000000,0.282828
pinky,96,0,0,0,65,31,0.830729,1.000000,0.322917
csplit,632,0,1,0,331,300,0.867880,1.000000,0.474684
fmt,112,0,0,0,72,40,0.839286,1.000000,0.357143
df,327,0,0,0,158,169,0.879205,1.000000,0.516820
join,162,0,0,0,104,58,0.839506,1.000000,0.358025
expr,573,0,1,0,292,280,0.871291,1.000000,0.488656
seq,156,0,0,0,78,78,0.875000,1.000000,0.500000
unexpand,101,0,0,0,61,40,0.849010,1.000000,0.396040


...

Varnode average compare score [0,1] (metatype=STRUCT)    0.998733
Varnodes fraction partially recovered                    1.000000
Varnodes fraction exactly recovered                      0.998311
dtype: float64

varnodes_metatype_UNION (optimization=0, stripped=False, debug=True)


,Ground truth varnodes (metatype=UNION),Decompiler varnodes matched @ level=NO_MATCH (metatype=UNION),Decompiler varnodes matched @ level=OVERLAP (metatype=UNION),Decompiler varnodes matched @ level=SUBSET (metatype=UNION),Decompiler varnodes matched @ level=ALIGNED (metatype=UNION),Decompiler varnodes matched @ level=MATCH (metatype=UNION),"Varnode average compare score [0,1] (metatype=UNION)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,0,0,0,0,0,0,NaN,NaN,NaN
nohup,0,0,0,0,0,0,NaN,NaN,NaN
pinky,0,0,0,0,0,0,NaN,NaN,NaN
csplit,0,0,0,0,0,0,NaN,NaN,NaN
fmt,0,0,0,0,0,0,NaN,NaN,NaN
df,0,0,0,0,0,0,NaN,NaN,NaN
join,0,0,0,0,0,0,NaN,NaN,NaN
expr,0,0,0,0,0,0,NaN,NaN,NaN
seq,0,0,0,0,0,0,NaN,NaN,NaN
unexpand,0,0,0,0,0,0,NaN,NaN,NaN


Varnode average compare score [0,1] (metatype=UNION)   NaN
Varnodes fraction partially recovered                  NaN
Varnodes fraction exactly recovered                    NaN
dtype: float64

,NO_MATCH,OVERLAP,SUBSET,ALIGNED,MATCH
INT,0.007421,0.000530,0.000000,0.00318,0.988868
FLOAT,0.000000,0.000000,0.000000,0.00000,1.000000
POINTER,0.009042,0.000000,0.000000,0.00000,0.990958
ARRAY,0.142857,0.002421,0.009685,0.00000,0.845036
STRUCT,0.000000,0.002959,0.000000,0.00000,0.997041


In [63]:
analyzed_opts_sets = (strip_opts, debug_opts)

for opts in (debug_opts,):
    for grp in varnodes_groups_metatypes:
        display(get_table(grp, opts))
        display(get_varnode_group_average_stats(grp))
    
    display(load_table("metatype_match_levels_ratios", opts))
# display_analyzed_tables(
#     decomposed_varnodes_groups,
#     (strip_opts,),
#     analyzed_only=True
# )

,Ground truth varnodes (metatype=INT),Decompiler varnodes matched @ level=NO_MATCH (metatype=INT),Decompiler varnodes matched @ level=OVERLAP (metatype=INT),Decompiler varnodes matched @ level=SUBSET (metatype=INT),Decompiler varnodes matched @ level=ALIGNED (metatype=INT),Decompiler varnodes matched @ level=MATCH (metatype=INT),"Varnode average compare score [0,1] (metatype=INT)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,392,0,0,0,0,392,1.000000,1.000000,1.000000
nohup,99,0,0,0,0,99,1.000000,1.000000,1.000000
pinky,96,0,0,0,0,96,1.000000,1.000000,1.000000
csplit,632,0,1,0,6,625,0.996440,1.000000,0.988924
fmt,112,0,0,0,0,112,1.000000,1.000000,1.000000
df,327,0,0,0,0,327,1.000000,1.000000,1.000000
join,162,0,0,0,0,162,1.000000,1.000000,1.000000
expr,573,0,1,0,6,566,0.996073,1.000000,0.987784
seq,156,0,0,0,0,156,1.000000,1.000000,1.000000
unexpand,101,0,0,0,0,101,1.000000,1.000000,1.000000


Varnode average compare score [0,1] (metatype=INT)    0.996141
Varnodes fraction partially recovered                 0.996609
Varnodes fraction exactly recovered                   0.995153
dtype: float64

,Ground truth varnodes (metatype=FLOAT),Decompiler varnodes matched @ level=NO_MATCH (metatype=FLOAT),Decompiler varnodes matched @ level=OVERLAP (metatype=FLOAT),Decompiler varnodes matched @ level=SUBSET (metatype=FLOAT),Decompiler varnodes matched @ level=ALIGNED (metatype=FLOAT),Decompiler varnodes matched @ level=MATCH (metatype=FLOAT),"Varnode average compare score [0,1] (metatype=FLOAT)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,2,0,0,0,0,2,1.0,1.0,1.0
nohup,0,0,0,0,0,0,NaN,NaN,NaN
pinky,0,0,0,0,0,0,NaN,NaN,NaN
csplit,0,0,0,0,0,0,NaN,NaN,NaN
fmt,0,0,0,0,0,0,NaN,NaN,NaN
df,13,0,0,0,0,13,1.0,1.0,1.0
join,0,0,0,0,0,0,NaN,NaN,NaN
expr,0,0,0,0,0,0,NaN,NaN,NaN
seq,10,0,0,0,0,10,1.0,1.0,1.0
unexpand,0,0,0,0,0,0,NaN,NaN,NaN


Varnode average compare score [0,1] (metatype=FLOAT)    1.0
Varnodes fraction partially recovered                   1.0
Varnodes fraction exactly recovered                     1.0
dtype: float64

,Ground truth varnodes (metatype=POINTER),Decompiler varnodes matched @ level=NO_MATCH (metatype=POINTER),Decompiler varnodes matched @ level=OVERLAP (metatype=POINTER),Decompiler varnodes matched @ level=SUBSET (metatype=POINTER),Decompiler varnodes matched @ level=ALIGNED (metatype=POINTER),Decompiler varnodes matched @ level=MATCH (metatype=POINTER),"Varnode average compare score [0,1] (metatype=POINTER)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,150,0,0,0,0,150,1.000000,1.000000,1.000000
nohup,40,0,0,0,0,40,1.000000,1.000000,1.000000
pinky,57,0,0,0,0,57,1.000000,1.000000,1.000000
csplit,280,0,0,0,0,280,1.000000,1.000000,1.000000
fmt,52,0,0,0,0,52,1.000000,1.000000,1.000000
df,244,0,0,0,0,244,1.000000,1.000000,1.000000
join,66,0,0,0,0,66,1.000000,1.000000,1.000000
expr,267,0,0,0,0,267,1.000000,1.000000,1.000000
seq,81,0,0,0,0,81,1.000000,1.000000,1.000000
unexpand,36,0,0,0,0,36,1.000000,1.000000,1.000000


...

,Ground truth varnodes (metatype=STRUCT),Decompiler varnodes matched @ level=NO_MATCH (metatype=STRUCT),Decompiler varnodes matched @ level=OVERLAP (metatype=STRUCT),Decompiler varnodes matched @ level=SUBSET (metatype=STRUCT),Decompiler varnodes matched @ level=ALIGNED (metatype=STRUCT),Decompiler varnodes matched @ level=MATCH (metatype=STRUCT),"Varnode average compare score [0,1] (metatype=STRUCT)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,32,0,0,0,0,32,1.00000,1.0,1.000000
nohup,10,0,0,0,0,10,1.00000,1.0,1.000000
pinky,12,0,0,0,0,12,1.00000,1.0,1.000000
csplit,56,0,0,0,0,56,1.00000,1.0,1.000000
fmt,10,0,0,0,0,10,1.00000,1.0,1.000000
df,37,0,1,0,0,36,0.97973,1.0,0.972973
join,14,0,0,0,0,14,1.00000,1.0,1.000000
expr,59,0,0,0,0,59,1.00000,1.0,1.000000
seq,17,0,0,0,0,17,1.00000,1.0,1.000000
unexpand,10,0,0,0,0,10,1.00000,1.0,1.000000


Varnode average compare score [0,1] (metatype=STRUCT)    0.998733
Varnodes fraction partially recovered                    1.000000
Varnodes fraction exactly recovered                      0.998311
dtype: float64

,Ground truth varnodes (metatype=UNION),Decompiler varnodes matched @ level=NO_MATCH (metatype=UNION),Decompiler varnodes matched @ level=OVERLAP (metatype=UNION),Decompiler varnodes matched @ level=SUBSET (metatype=UNION),Decompiler varnodes matched @ level=ALIGNED (metatype=UNION),Decompiler varnodes matched @ level=MATCH (metatype=UNION),"Varnode average compare score [0,1] (metatype=UNION)",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,0,0,0,0,0,0,NaN,NaN,NaN
nohup,0,0,0,0,0,0,NaN,NaN,NaN
pinky,0,0,0,0,0,0,NaN,NaN,NaN
csplit,0,0,0,0,0,0,NaN,NaN,NaN
fmt,0,0,0,0,0,0,NaN,NaN,NaN
df,0,0,0,0,0,0,NaN,NaN,NaN
join,0,0,0,0,0,0,NaN,NaN,NaN
expr,0,0,0,0,0,0,NaN,NaN,NaN
seq,0,0,0,0,0,0,NaN,NaN,NaN
unexpand,0,0,0,0,0,0,NaN,NaN,NaN


Varnode average compare score [0,1] (metatype=UNION)   NaN
Varnodes fraction partially recovered                  NaN
Varnodes fraction exactly recovered                    NaN
dtype: float64

,NO_MATCH,OVERLAP,SUBSET,ALIGNED,MATCH
INT,0.001327,0.001280,0.000000,0.001896,0.995497
FLOAT,0.000000,0.000000,0.000000,0.000000,1.000000
POINTER,0.001579,0.000000,0.000000,0.000105,0.998316
ARRAY,0.026916,0.007755,0.010949,0.000000,0.954380
STRUCT,0.000000,0.003935,0.000000,0.000000,0.996065


In [64]:
cmp = load_cmp(prog_from_progname("cksum"), debug_opts)
truth = sum([ varnode.get_size() for varnode in varnodes_truth(cmp) ])
missed = sum([ varnode.get_size() for varnode in varnodes_missed(cmp) ])
overlapped = varnode_compare_records_matched_at_level(cmp, VarnodeCompareLevel.OVERLAP)
for varnode in varnodes_missed(cmp):
    print(varnode.get_var().get_parent_function().get_name())

# for record in overlapped:
#     print(record.get_varnode().get_var().get_parent_function().get_name())
# print(overlapped)

cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul